In [2]:
import vtk
import glob
from tqdm import tqdm_notebook as tqdm
from scipy.misc import imread
import warnings
warnings.filterwarnings("ignore")
import numpy as np

In [3]:
cd /home/xupeng/projects/membrane/

/home/xupeng/projects/membrane


In [4]:
ls data/toy/

KK_mask1_em_export_s000.png  KK_mask1_em_export_s005.png
KK_mask1_em_export_s001.png  KK_mask1_em_export_s006.png
KK_mask1_em_export_s002.png  KK_mask1_em_export_s007.png
KK_mask1_em_export_s003.png  KK_mask1_em_export_s008.png
KK_mask1_em_export_s004.png  KK_mask1_em_export_s009.png


In [5]:
from vtk.util import numpy_support

pngfiles = glob.glob('data/toy/*.png')

png_reader = vtk.vtkPNGReader()

In [7]:
png_reader

(vtkIOImagePython.vtkPNGReader)0x7f0139cb6648

In [6]:
pngfiles[0]

'data/toy/KK_mask1_em_export_s001.png'

In [35]:
imread('data/mask1/png/KK_mask1_em_export_s128.png').shape

(1413, 2469, 3)

In [36]:
png_reader.SetFileName(pngfiles[0])
png_reader.GetOutput().GetDimensions()

(0, 0, 0)

In [39]:
x,y = 1413,2469
data_3d = np.zeros([x,y,len(pngfiles)])

In [44]:
for i in tqdm(range(len(pngfiles))):
    png_reader.SetFileName(pngfiles[0])
    png_reader.Update()
    img_data = png_reader.GetOutput()
    data_3D[:,:,i] = numpy_support.vtk_to_numpy(img_data)

#save your 3D numpy array out.
data_3Dvtk = numpy_support.numpy_to_vtk(data_3D)

AttributeError: 'vtkCommonDataModelPython.vtkImageData' object has no attribute 'GetDataType'

In [8]:
import vtk
from vtk.util.misc import vtkGetDataRoot
VTK_DATA_ROOT = vtkGetDataRoot()

In [9]:
VTK_DATA_ROOT

'../../../../VTKData'

In [ ]:
reader = vtk.vtkImageReader2()
reader.SetFilePrefix(VTK_DATA_ROOT + "/Data/headsq/quarter")
reader.SetDataExtent(0, 63, 0, 63, 1, 93)
reader.SetDataSpacing(3.2, 3.2, 1.5)
reader.SetDataOrigin(0.0, 0.0, 0.0)
reader.SetDataScalarTypeToUnsignedShort()
reader.UpdateWholeExtent()

In [ ]:
resliceFilter = vtk.vtkImageReslice()
resliceFilter.SetInput(image)
resliceFilter.SetOutputSpacing(0.5, 0.5, 0.5)
resliceFilter.SetInterpolationModeToCubic()
resliceFilter.Update()
imageResampled = resliceFilter.GetOutput()

In [ ]:
# Start by loading some data.
reader = vtk.vtkImageReader2()
reader.SetFilePrefix("data/mask1/png/")

In [3]:

reader.SetDataExtent(0, 63, 0, 63, 1, 93)
reader.SetDataSpacing(3.2, 3.2, 1.5)
reader.SetDataOrigin(0.0, 0.0, 0.0)
reader.SetDataScalarTypeToUnsignedShort()
reader.UpdateWholeExtent()

In [4]:
reader

(vtkIOImagePython.vtkImageReader2)0x7f86c828ffa8

In [5]:
# Calculate the center of the volume
reader.Update()

In [6]:
reader.GetExecutive().GetWholeExtent(reader.GetOutputInformation(0))

(0, 63, 0, 63, 1, 93)

In [ ]:
(xMin, xMax, yMin, yMax, zMin, zMax) = reader.GetExecutive().GetWholeExtent(reader.GetOutputInformation(0))
(xSpacing, ySpacing, zSpacing) = reader.GetOutput().GetSpacing()
(x0, y0, z0) = reader.GetOutput().GetOrigin()

center = [x0 + xSpacing * 0.5 * (xMin + xMax),
          y0 + ySpacing * 0.5 * (yMin + yMax),
          z0 + zSpacing * 0.5 * (zMin + zMax)]

# Matrices for axial, coronal, sagittal, oblique view orientations
axial = vtk.vtkMatrix4x4()
axial.DeepCopy((1, 0, 0, center[0],
                0, 1, 0, center[1],
                0, 0, 1, center[2],
                0, 0, 0, 1))

coronal = vtk.vtkMatrix4x4()
coronal.DeepCopy((1, 0, 0, center[0],
                  0, 0, 1, center[1],
                  0,-1, 0, center[2],
                  0, 0, 0, 1))

sagittal = vtk.vtkMatrix4x4()
sagittal.DeepCopy((0, 0,-1, center[0],
                   1, 0, 0, center[1],
                   0,-1, 0, center[2],
                   0, 0, 0, 1))

oblique = vtk.vtkMatrix4x4()
oblique.DeepCopy((1, 0, 0, center[0],
                  0, 0.866025, -0.5, center[1],
                  0, 0.5, 0.866025, center[2],
                  0, 0, 0, 1))

# Extract a slice in the desired orientation
reslice = vtk.vtkImageReslice()
reslice.SetInputConnection(reader.GetOutputPort())
reslice.SetOutputDimensionality(2)
reslice.SetResliceAxes(sagittal)
reslice.SetInterpolationModeToLinear()

# Create a greyscale lookup table
table = vtk.vtkLookupTable()
table.SetRange(0, 2000) # image intensity range
table.SetValueRange(0.0, 1.0) # from black to white
table.SetSaturationRange(0.0, 0.0) # no color saturation
table.SetRampToLinear()
table.Build()

# Map the image through the lookup table
color = vtk.vtkImageMapToColors()
color.SetLookupTable(table)
color.SetInputConnection(reslice.GetOutputPort())

# Display the image
actor = vtk.vtkImageActor()
actor.GetMapper().SetInputConnection(color.GetOutputPort())

renderer = vtk.vtkRenderer()
renderer.AddActor(actor)

window = vtk.vtkRenderWindow()
window.AddRenderer(renderer)

# Set up the interaction
interactorStyle = vtk.vtkInteractorStyleImage()
interactor = vtk.vtkRenderWindowInteractor()
interactor.SetInteractorStyle(interactorStyle)
window.SetInteractor(interactor)
window.Render()

# Create callbacks for slicing the image
actions = {}
actions["Slicing"] = 0

def ButtonCallback(obj, event):
    if event == "LeftButtonPressEvent":
        actions["Slicing"] = 1
    else:
        actions["Slicing"] = 0

def MouseMoveCallback(obj, event):
    (lastX, lastY) = interactor.GetLastEventPosition()
    (mouseX, mouseY) = interactor.GetEventPosition()
    if actions["Slicing"] == 1:
        deltaY = mouseY - lastY
        reslice.Update()
        sliceSpacing = reslice.GetOutput().GetSpacing()[2]
        matrix = reslice.GetResliceAxes()
        # move the center point that we are slicing through
        center = matrix.MultiplyPoint((0, 0, sliceSpacing*deltaY, 1))
        matrix.SetElement(0, 3, center[0])
        matrix.SetElement(1, 3, center[1])
        matrix.SetElement(2, 3, center[2])
        window.Render()
    else:
        interactorStyle.OnMouseMove()


interactorStyle.AddObserver("MouseMoveEvent", MouseMoveCallback)
interactorStyle.AddObserver("LeftButtonPressEvent", ButtonCallback)
interactorStyle.AddObserver("LeftButtonReleaseEvent", ButtonCallback)

# Start interaction
interactor.Start()